# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import kyurem
from kyurem.core.idom_loader import reload_bundle
from kyurem.core.widget import WidgetModel

kyurem_client: 0.0.0


In [5]:
from kyurem import Service
service = Service(kh = 'indeed.v0.0.1:612e3403')

## Load data 

In [17]:
import pandas as pd
df1 = pd.read_csv("indeed.v0.0.1_612e3403_sampled.csv")
corpus_data = df1.to_dict('records')

df2 = pd.read_csv("indeed.v0.0.1_612e3403_merged.csv")
merge_data = df2.to_dict('records')

In [18]:
service.load_corpus(corpus_data, 'parent_title', 'context', 'child_title')

In [19]:
service.load_merge_data(merge_data, 'corpus_entity', 'node_label', 'node_uuid', 'node_title')

In [20]:
decision_list = ["Accept", "Reject", "Defer"]

In [22]:
mergedata = service.get_merge_data()["rows"]

# Create Widget

In [28]:
from kyurem import MergeVerifier


def init(state): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "corpus": None,
        "mergedata":mergedata,
    }

def focus(state, row, panel):
    # node can be in the following format:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    
    node = { "node_label": row["node_label"], "node_property": "uuid", "node_property_value": row["node_uuid"] }
    entity = node = { "node_label": row["node_label"], "node_property": "title", "node_property_value": row["entity"] }
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    datatable = service.get_annotated_corpus(node)
    
    data["subgraph"] = result["schema"]
    # TODO: Move conversion code into service
    if datatable["rows"]:
        data["corpus"] = datatable["rows"]
        data["highlight"] = datatable["highlight"]
    else:
        data["datatable"] = None
        data["highlight"] = None
    return data
  
# Create widget 
reload_bundle()
merge_verifier = MergeVerifier({
        "init": init,
        "focus": focus
    }, mergedata)

merge_verifier.show()

component(109b3dc40, self=<kyurem.widgets.WidgetWithHistory.WidgetWithHistory object at 0x11aa957c0>)

LayoutWidget(Layout(component(109b3dc40, self=<kyurem.widgets.WidgetWithHistory.WidgetWithHistory object at 0x…

## Select a node_title: Get node neighborhood

In [ ]:
selected_uuid = "0ec6aae471d34b0c8cc79eb223b120ad"
node = {"node_label":"attribute", "node_property":"uuid","node_property_value":selected_uuid}
service.get_node_neighborhood(node)

## Select an entity: Get Corpus Table

In [ ]:
selectedentity = ""
node = {"node_label":None, "node_property":"title","node_property_value":entity}
contextable = service.get_annotated_corpus(node)